## Install dependencies

In [72]:
from datetime import datetime, timedelta, timezone
import pandas as pd
import numpy as np
import requests
import os
import logging
import sys
import optuna
import joblib
import xgboost as xgb

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from datetime import datetime, timedelta
from ta import add_all_ta_features

## Variables

In [ ]:
current_storage = os.path.join("d:", "data")
vn30f_storage = os.path.join(current_storage, 'vn30f')

In [57]:
start_time = 0
now_time = 9999999999
symbol = 'VN30F1M'
rolling_window = 1 # Số phút muốn dự đoán tiếp theo

In [62]:
# Add stream handler of stdout to show the messages
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "btc1-study"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)

## Formulas

In [5]:
def choose_position(roi, trade_threshold = 0.01):
    pos =0
    # Predict position base on change in future
    if roi > trade_threshold:
        pos = 1
    elif roi < -trade_threshold:
        pos = -1
    else:
        pos = 0

    return pos

In [6]:
def backtest_position_ps(position, price, percentage, periods):
    # Shift positions to align with future price changes and handle NaN by filling with 0
    pos = pd.Series(position, index=pd.Series(price).index).shift(1).fillna(0)
    pos = pd.Series(pos).rolling(periods).sum() #pos for 10 hour predict

    price_array = pd.Series(price).shift(1).fillna(0)

    pos_diff = pos.diff()
    fee = pos_diff*price_array*0.05*0.01

    # Calculate price changes over the given periods
    ch = pd.Series(price) - price_array

    # Calculate total PnL
    total_pnl = pos*ch - fee
    return total_pnl

In [7]:
def calculate_sharpe_ratio(pnl):
    pnl = np.diff(pnl)
    std = np.std(pnl) if np.std(pnl) != 0 else 0.001
    sharpe = np.mean(pnl)/std*np.sqrt(252)
    return sharpe

In [8]:
def sharpe_for_vn30f(y_pred, y_price, trade_threshold, fee_perc, periods):

    # Predict position base on change in future
    pos = [choose_position(roi, trade_threshold) for roi in y_pred]
    pos = np.array(pos)

    # Calculate PNL
    pnl = backtest_position_ps(pos, y_price, fee_perc, periods)
    pnl = np.cumsum(pnl)

    # Standardalize PNL to date
    daily_pnl = [pnl.iloc[i] for i in range(0, len(pnl), 24)]
    daily_pnl = pd.Series(daily_pnl).fillna(0)

    # Calculate Sharpe
    sharpe = calculate_sharpe_ratio(daily_pnl)

    return pos, pnl, daily_pnl, sharpe

In [9]:
def calculate_hitrate(pos_predict, pos_true):
    if len(pos_predict) != len(pos_true):
        raise ValueError("Độ dài của hai mảng không khớp")

    # Tính số lượng dự đoán đúng (các phần tử tương ứng giống nhau)
    correct_predictions = np.sum(pos_predict == pos_true)

    # Tính tỷ lệ hit rate
    hit_rate_value = correct_predictions / len(pos_predict)

    return hit_rate_value

# Function for data

In [10]:
def scale_data(data):
    scaler = StandardScaler()
    scaler.fit(data)
    data=pd.DataFrame(scaler.transform(data), index=data.index, columns=data.columns)

    return data

In [65]:
def split_data(data):
    """
    Hàm này chia dữ liệu thành 2 phần: tập huấn luyện và tập hold out.

    Args:
    data (pandas.DataFrame): DataFrame chứa dữ liệu cần chia.

    Returns:
    pandas.DataFrame: DataFrame chứa dữ liệu tập huấn luyện.
    pandas.DataFrame: DataFrame chứa dữ liệu tập giữ lại.
    """
    # Chia dữ liệu thành 3 phần
    new_part = np.array_split(data, 3)

    # Access each part individually
    hold_out = new_part[2]
    train_data = pd.concat([new_part[0], new_part[1]], axis=0)

    return train_data, hold_out

def split_optuna_data(data):
    """
    Hàm này chia dữ liệu thành các tập train và test để sử dụng trong quá trình tối ưu hóa bằng Optuna.

    Args:
    data (pandas.DataFrame): DataFrame chứa dữ liệu cần chia.

    Returns:
    pandas.DataFrame: DataFrame chứa dữ liệu train (đã được chuẩn hóa).
    pandas.DataFrame: DataFrame chứa dữ liệu test (đã được chuẩn hóa).
    pandas.Series: Series chứa nhãn tương ứng với dữ liệu train.
    pandas.Series: Series chứa nhãn tương ứng với dữ liệu test.
    """
    # Chia dữ liệu thành tập train và tập hold out
    train_data, _ = split_data(data)
    
    # Loại bỏ các cột không cần thiết
    if 'Open' in train_data.columns:
        optuna_data = train_data.drop(['Open','High','Low','Close','Volume', 'Return', 'Date', 'time'], axis=1)
    else:
        optuna_data = train_data.drop(['Close', 'Return', 'Date', 'time'], axis=1)
    
    # Chuẩn hóa dữ liệu
    optuna_data = scale_data(optuna_data)
    
    # Chia dữ liệu thành tập train và tập test
    X_train, X_valid, y_train, y_valid = train_test_split(optuna_data, train_data['Return'], test_size=0.5, shuffle=False)

    return X_train, X_valid, y_train, y_valid, train_data

## Get data

In [11]:
def get_vn30f(start_time, now_time, symbol):
    def vn30f():
            return requests.get(f"https://services.entrade.com.vn/chart-api/chart?from={start_time}&resolution=1&symbol={symbol}&to={now_time}").json()
    vn30fm = pd.DataFrame(vn30f()).iloc[:,:6]
    vn30fm['t'] = vn30fm['t'].astype(int).apply(lambda x: datetime.utcfromtimestamp(x) + timedelta(hours = 7))
    vn30fm.columns = ['Date','Open','High','Low','Close','Volume']
    ohlc_dict = {                                                                                                             
        'Open': 'first',                                                                                                    
        'High': 'max',                                                                                                       
        'Low': 'min',                                                         
        'Close': 'last',                                                                                                    
        'Volume': 'sum',}
    vn30fm = pd.DataFrame(vn30f()).iloc[:,:6]
    vn30fm['t'] = vn30fm['t'].astype(int).apply(lambda x: datetime.utcfromtimestamp(x) + timedelta(hours = 7))
    vn30fm.columns = ['Date','Open','High','Low','Close','Volume']
    dt_object = datetime.utcfromtimestamp(start_time) + timedelta(hours = 7)
    now_object = datetime.utcfromtimestamp(now_time) + timedelta(hours = 7)

    print(f'===> Data {symbol} from {dt_object} to {now_object} has been appended ')
    
    return vn30fm

In [12]:
df = get_vn30f(start_time, now_time, symbol)

===> Data VN30F1M from 1970-01-01 07:00:00 to 2286-11-21 00:46:39 has been appended 


In [50]:
data = df.copy()

## Preprocess

Thêm phần xử lý outlier ở đây

In [51]:
def process_data(data):
    data.set_index('Date', inplace =True)
    data.columns = ['Open','High','Low','Close','Volume']

    data['Date'] = [str(i)[:10] for i in data.index]
    data['time'] = [str(i)[11:] for i in data.index]

    data = data[~data.index.duplicated(keep='first')] # Handling duplicate
    data_model = data.pivot(index = 'Date', columns = 'time', values = ['Open','High','Low','Close','Volume']).ffill(axis = 1).stack().reset_index() # Handling missing values
    # data_model.columns = ['Date','time','Close']

    return data_model

In [52]:
data = process_data(data)

In [53]:
data

,Date,time,Open,High,Low,Close,Volume
0,2018-08-13,09:00:00,943.5,943.5,942.9,942.9,975.0
1,2018-08-13,09:01:00,943.0,943.1,942.9,943.1,220.0
2,2018-08-13,09:02:00,943.0,943.6,943.0,943.5,121.0
3,2018-08-13,09:03:00,943.3,943.4,943.3,943.4,135.0
4,2018-08-13,09:04:00,943.2,943.2,943.0,943.1,361.0
...,...,...,...,...,...,...,...
321831,2024-05-31,14:27:00,1269.4,1269.7,1268.6,1268.8,1707.0
321832,2024-05-31,14:28:00,1268.7,1269.4,1268.7,1269.4,1805.0
321833,2024-05-31,14:29:00,1269.2,1269.4,1268.3,1268.3,2103.0
321834,2024-05-31,14:30:00,1268.2,1268.6,1268.2,1268.6,224.0


## Create features

In [54]:
def generate_features(df, shift=1):
    """
    Hàm này tạo ra các features mới từ dữ liệu cổ phiếu.

    Args:
    df (pandas.DataFrame): DataFrame chứa dữ liệu cần tạo features, có các cột 'open', 'high', 'low', 'close' và 'volume'.

    Returns:
    pandas.DataFrame: DataFrame đã được mở rộng với các features mới.
    """

    # Thêm tất cả các đặc trưng kỹ thuật từ thư viện TA-Lib
    df_shift = df.shift(shift)
    df_shift = add_all_ta_features(df_shift, open="Open", high="High", low="Low", close="Close", volume="Volume")

    # Drop the original OHLCV columns from the shifted DataFrame
    df_shift = df_shift.drop(columns=['Date','time', 'Open','High','Low','Close','Volume'])

    df = pd.concat([df, df_shift], axis=1)

    ### Ví dụ các features thêm tay
    # Tính phần trăm thay đổi của giá đóng cửa trong 5 phút
    df['Trend_5min'] = df['Close'].shift(shift).pct_change(periods=5)

    # Tính độ lệch chuẩn của giá đóng cửa trong 30 phút
    df['Std_Rolling_30min'] = df['Close'].shift(shift).rolling(window=30, min_periods=1).std()

    # Tính độ lệch của giá đóng cửa so với trung bình động 30 phút
    ma_30min = df['Close'].shift(shift).rolling(window=30, min_periods=1).mean()
    df['Close_Minus_MA_30min'] = df['Close'].shift(shift) - ma_30min

    # Tính trung bình động 10 phút của giá đóng cửa
    df['SMA_10'] = df['Close'].shift(shift).rolling(window=10, min_periods=1).mean()

    # Identify numeric columns
    numeric_cols = df.select_dtypes(include=[np.number]).columns

    # Replace infinite values with NaN in numeric columns
    df[numeric_cols] = df[numeric_cols].replace([np.inf, -np.inf], np.nan)

    # Điền các giá trị NaN với 0
    df = df.fillna(0)

    return df

In [55]:
data  = generate_features(data)

Thêm các feature khác, ví dụ giá vàng (xem xét phù hợp với thời gian trong data)

# Add Predict Features

In [61]:
data['Return'] = (data['Close'].shift(-rolling_window) - data['Close'])/data['Close']
data = data.fillna(0)
data = data.drop(index=0)
data

,Date,time,Open,High,Low,Close,Volume,volume_adi,volume_obv,volume_cmf,...,momentum_pvo_hist,momentum_kama,others_dr,others_dlr,others_cr,Trend_5min,Std_Rolling_30min,Close_Minus_MA_30min,SMA_10,Return
1,2018-08-13,09:01:00,943.0,943.1,942.9,943.1,220.0,-975.000000,975.0,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,942.900000,0.000424
2,2018-08-13,09:02:00,943.0,943.6,943.0,943.5,121.0,-755.000000,1195.0,0.000000,...,0.000000,0.000000,0.021211,0.021209,0.0,0.000000,0.141421,0.100000,943.000000,-0.000106
3,2018-08-13,09:03:00,943.3,943.4,943.3,943.4,135.0,-674.333333,1316.0,0.000000,...,0.000000,0.000000,0.042413,0.042404,0.0,0.000000,0.305505,0.333333,943.166667,-0.000318
4,2018-08-13,09:04:00,943.2,943.2,943.0,943.1,361.0,-539.333333,1181.0,0.000000,...,0.000000,0.000000,-0.010599,-0.010599,0.0,0.000000,0.275379,0.175000,943.225000,-0.000106
5,2018-08-13,09:05:00,943.1,943.1,942.9,943.0,343.0,-539.333333,820.0,0.000000,...,0.000000,0.000000,-0.031800,-0.031805,0.0,0.000000,0.244949,-0.100000,943.200000,0.000106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321831,2024-05-31,14:27:00,1269.4,1269.7,1268.6,1268.8,1707.0,-265438.223300,16175491.4,0.046359,...,-11.227438,1269.377569,-0.062977,-0.062997,0.0,-0.000158,1.687357,-0.620000,1270.160000,0.000473
321832,2024-05-31,14:28:00,1268.7,1269.4,1268.7,1269.4,1805.0,-266524.496028,16173784.4,0.120624,...,-6.297804,1269.338726,-0.055140,-0.055155,0.0,-0.001102,1.680093,-1.226667,1269.940000,-0.000867
321833,2024-05-31,14:29:00,1269.2,1269.4,1268.3,1268.3,2103.0,-264719.496028,16175589.4,0.130776,...,-2.473416,1269.340451,0.047289,0.047278,0.0,-0.000787,1.656905,-0.553333,1269.820000,0.000237
321834,2024-05-31,14:30:00,1268.2,1268.6,1268.2,1268.6,224.0,-266822.496028,16173486.4,0.019718,...,1.278459,1269.311175,-0.086655,-0.086693,0.0,-0.001181,1.649577,-1.540000,1269.700000,0.001104


# Feature Engineering

In [67]:
def drop_high_corr_columns(df, threshold=0.6):
    ohlcv_columns = {'Date','time', 'Open','High','Low','Close','Volume'}

     # Identify non-OHLCV and non-date/time columns
    non_corr_columns = [col for col in df.columns if col not in ohlcv_columns]

    # Compute the correlation matrix only for non-OHLCV and non-date/time columns
    corr_matrix = df[non_corr_columns].corr().abs()

    # Create a set to keep track of columns to drop
    to_drop = set()

    # Iterate over the upper triangle of the correlation matrix
    for i in range(len(corr_matrix.columns)):
        for j in range(i + 1, len(corr_matrix.columns)):
            if corr_matrix.iloc[i, j] > threshold:
                # If the correlation is higher than the threshold, mark the column with the higher index to drop
                col_to_drop = corr_matrix.columns[j]
                if col_to_drop not in ohlcv_columns:
                    to_drop.add(col_to_drop)

    # Drop the columns from the DataFrame
    df_dropped = df.drop(columns=to_drop)
    return df_dropped

data = drop_high_corr_columns(data)

In [69]:
def objective(trial, X_train, X_valid, y_train, y_valid, y_price):

    # Select features based on Optuna's suggestions
    selected_features = []
    for col in X_train.columns:
        if trial.suggest_categorical(col, [0, 1]):
            selected_features.append(col)

    for t in trial.study.trials:
        if t.state != optuna.trial.TrialState.COMPLETE:
            continue

    # trade_threshold = trial.suggest_float('trade_threshold', 0.008, 0.08, step=0.0004)
    trade_threshold  = 0.005

    # Use only the selected features in training
    X_train_selected = X_train[selected_features]
    X_valid_selected = X_valid[selected_features]

    # Train the model
    model = xgb.XGBRegressor()
    model.fit(X_train_selected, y_train)

    y_pred_train = model.predict(X_train_selected)
    y_pred_valid = model.predict(X_valid_selected)

    _, pos_is, _, sharpe_is = sharpe_for_vn30f(y_pred_train, y_price[:len(y_pred_train)], trade_threshold=trade_threshold, fee_perc=0.01, periods=10)
    _, pos_os, _, sharpe_oos = sharpe_for_vn30f(y_pred_valid, y_price[len(y_pred_train):], trade_threshold=trade_threshold, fee_perc=0.01, periods=10)

    print('Trade times in sample:',len(pos_is[pos_is != 0]), 'Trade times out sample:',len(pos_os[pos_os != 0]))

    return sharpe_oos, abs((abs(sharpe_is / sharpe_oos))-1)

X_train, X_valid, y_train, y_valid, train_data = split_optuna_data(data)

In [73]:
"""## Define number of trials (no 2)"""

# Create a study object and optimize the objective function
study = optuna.create_study(directions=['maximize', 'minimize'])
unique_trials = 50

while unique_trials > len(set(str(t.params) for t in study.trials)):
    study.optimize(lambda trial: objective(trial, X_train, X_valid, y_train, y_valid, train_data['Close']), n_trials=1)
    study.trials_dataframe().sort_values('values_0').to_csv('feature_trials.csv')
    joblib.dump(study, 'abmodel.pkl')
    
trials = study.trials
trials.sort(key=lambda trial: trial.values, reverse=True)
study.trials_dataframe().sort_values('values_0', ascending=False).duplicated().sum()

"""## Checking part for fixing bug"""
df =study.trials_dataframe()

df.to_csv('save_trials.csv')

[I 2024-06-02 22:53:51,945] A new study created in memory with name: no-name-f2ae8cf5-21d3-4de5-bf09-7e2ef6b7b676


A new study created in memory with name: no-name-f2ae8cf5-21d3-4de5-bf09-7e2ef6b7b676


[I 2024-06-02 22:53:52,793] Trial 0 finished with values: [0.2399149404562337, 1.3250724034610033] and parameters: {'volume_adi': 0, 'volume_obv': 0, 'volume_cmf': 0, 'volume_fi': 1, 'volume_em': 0, 'volume_sma_em': 0, 'volume_mfi': 1, 'volatility_bbw': 1, 'volatility_bbhi': 0, 'volatility_bbli': 1, 'volatility_kcp': 1, 'trend_macd': 0, 'trend_mass_index': 1, 'trend_dpo': 0, 'trend_adx': 0, 'trend_aroon_up': 0, 'trend_aroon_down': 1, 'trend_psar_up': 0, 'trend_psar_down': 1, 'trend_psar_up_indicator': 0, 'trend_psar_down_indicator': 1, 'momentum_pvo': 0, 'momentum_pvo_hist': 0, 'others_dr': 1, 'others_cr': 0}. 


Trade times in sample: 12129 Trade times out sample: 106962
Trial 0 finished with values: [0.2399149404562337, 1.3250724034610033] and parameters: {'volume_adi': 0, 'volume_obv': 0, 'volume_cmf': 0, 'volume_fi': 1, 'volume_em': 0, 'volume_sma_em': 0, 'volume_mfi': 1, 'volatility_bbw': 1, 'volatility_bbhi': 0, 'volatility_bbli': 1, 'volatility_kcp': 1, 'trend_macd': 0, 'trend_mass_index': 1, 'trend_dpo': 0, 'trend_adx': 0, 'trend_aroon_up': 0, 'trend_aroon_down': 1, 'trend_psar_up': 0, 'trend_psar_down': 1, 'trend_psar_up_indicator': 0, 'trend_psar_down_indicator': 1, 'momentum_pvo': 0, 'momentum_pvo_hist': 0, 'others_dr': 1, 'others_cr': 0}. 


[I 2024-06-02 22:53:53,734] Trial 1 finished with values: [-0.30780333748207583, 2.0050306868415215] and parameters: {'volume_adi': 0, 'volume_obv': 1, 'volume_cmf': 1, 'volume_fi': 1, 'volume_em': 1, 'volume_sma_em': 1, 'volume_mfi': 0, 'volatility_bbw': 0, 'volatility_bbhi': 0, 'volatility_bbli': 0, 'volatility_kcp': 1, 'trend_macd': 1, 'trend_mass_index': 0, 'trend_dpo': 1, 'trend_adx': 0, 'trend_aroon_up': 0, 'trend_aroon_down': 1, 'trend_psar_up': 0, 'trend_psar_down': 0, 'trend_psar_up_indicator': 1, 'trend_psar_down_indicator': 1, 'momentum_pvo': 0, 'momentum_pvo_hist': 1, 'others_dr': 0, 'others_cr': 1}. 


...

Trade times in sample: 94845 Trade times out sample: 106963
Trial 47 finished with values: [-0.24310167479417658, 2.824932528330221] and parameters: {'volume_adi': 1, 'volume_obv': 1, 'volume_cmf': 1, 'volume_fi': 1, 'volume_em': 1, 'volume_sma_em': 1, 'volume_mfi': 0, 'volatility_bbw': 0, 'volatility_bbhi': 1, 'volatility_bbli': 0, 'volatility_kcp': 0, 'trend_macd': 0, 'trend_mass_index': 0, 'trend_dpo': 0, 'trend_adx': 0, 'trend_aroon_up': 0, 'trend_aroon_down': 1, 'trend_psar_up': 1, 'trend_psar_down': 1, 'trend_psar_up_indicator': 1, 'trend_psar_down_indicator': 0, 'momentum_pvo': 1, 'momentum_pvo_hist': 1, 'others_dr': 1, 'others_cr': 1}. 


[I 2024-06-02 22:54:38,085] Trial 48 finished with values: [-0.44278225569874047, 0.7974884492074179] and parameters: {'volume_adi': 1, 'volume_obv': 1, 'volume_cmf': 1, 'volume_fi': 0, 'volume_em': 1, 'volume_sma_em': 0, 'volume_mfi': 1, 'volatility_bbw': 1, 'volatility_bbhi': 0, 'volatility_bbli': 1, 'volatility_kcp': 0, 'trend_macd': 1, 'trend_mass_index': 0, 'trend_dpo': 0, 'trend_adx': 0, 'trend_aroon_up': 1, 'trend_aroon_down': 0, 'trend_psar_up': 1, 'trend_psar_down': 1, 'trend_psar_up_indicator': 1, 'trend_psar_down_indicator': 1, 'momentum_pvo': 0, 'momentum_pvo_hist': 1, 'others_dr': 1, 'others_cr': 0}. 


Trade times in sample: 70445 Trade times out sample: 106963
Trial 48 finished with values: [-0.44278225569874047, 0.7974884492074179] and parameters: {'volume_adi': 1, 'volume_obv': 1, 'volume_cmf': 1, 'volume_fi': 0, 'volume_em': 1, 'volume_sma_em': 0, 'volume_mfi': 1, 'volatility_bbw': 1, 'volatility_bbhi': 0, 'volatility_bbli': 1, 'volatility_kcp': 0, 'trend_macd': 1, 'trend_mass_index': 0, 'trend_dpo': 0, 'trend_adx': 0, 'trend_aroon_up': 1, 'trend_aroon_down': 0, 'trend_psar_up': 1, 'trend_psar_down': 1, 'trend_psar_up_indicator': 1, 'trend_psar_down_indicator': 1, 'momentum_pvo': 0, 'momentum_pvo_hist': 1, 'others_dr': 1, 'others_cr': 0}. 


[I 2024-06-02 22:54:39,119] Trial 49 finished with values: [-0.2665902564118872, 2.3344237338188845] and parameters: {'volume_adi': 1, 'volume_obv': 0, 'volume_cmf': 1, 'volume_fi': 0, 'volume_em': 0, 'volume_sma_em': 1, 'volume_mfi': 1, 'volatility_bbw': 1, 'volatility_bbhi': 1, 'volatility_bbli': 1, 'volatility_kcp': 1, 'trend_macd': 1, 'trend_mass_index': 1, 'trend_dpo': 1, 'trend_adx': 0, 'trend_aroon_up': 1, 'trend_aroon_down': 0, 'trend_psar_up': 1, 'trend_psar_down': 0, 'trend_psar_up_indicator': 1, 'trend_psar_down_indicator': 1, 'momentum_pvo': 1, 'momentum_pvo_hist': 1, 'others_dr': 1, 'others_cr': 1}. 


Trade times in sample: 94845 Trade times out sample: 104038
Trial 49 finished with values: [-0.2665902564118872, 2.3344237338188845] and parameters: {'volume_adi': 1, 'volume_obv': 0, 'volume_cmf': 1, 'volume_fi': 0, 'volume_em': 0, 'volume_sma_em': 1, 'volume_mfi': 1, 'volatility_bbw': 1, 'volatility_bbhi': 1, 'volatility_bbli': 1, 'volatility_kcp': 1, 'trend_macd': 1, 'trend_mass_index': 1, 'trend_dpo': 1, 'trend_adx': 0, 'trend_aroon_up': 1, 'trend_aroon_down': 0, 'trend_psar_up': 1, 'trend_psar_down': 0, 'trend_psar_up_indicator': 1, 'trend_psar_down_indicator': 1, 'momentum_pvo': 1, 'momentum_pvo_hist': 1, 'others_dr': 1, 'others_cr': 1}. 
